In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_1859900/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        # self.discard_below_average(sort_by="r2")
        # self.discard_below_average(sort_by="r2_vt")
        # self.discard_high_standard_deviation()
        self.results_df.sort_values(by="mse")
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.1)
analize.Analize()
# analize.clean_folder(subfolder="dataset", extension="pkl")
# analize.clean_folder(subfolder="results", extension="xlsx")
# analize.clean_folder(subfolder="results", extension="txt")
# analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_10_49_9,0.997250,-0.200846,0.194173,0.989298,0.000325,0.712874,0.564498,0.000419,0.006569,0.018015,1.000534,0.018155,1602.066209,3876.021028,"Hidden Size=[36, 18], regularizer=0.02, learni..."
1,model_10_49_8,0.997214,-0.200750,0.194264,0.989377,0.000329,0.712817,0.564435,0.000416,0.006618,0.018133,1.000541,0.018274,1602.040105,3875.994924,"Hidden Size=[36, 18], regularizer=0.02, learni..."
2,model_10_49_7,0.997175,-0.200653,0.194351,0.989440,0.000333,0.712760,0.564374,0.000413,0.006667,0.018256,1.000549,0.018398,1602.012975,3875.967794,"Hidden Size=[36, 18], regularizer=0.02, learni..."
3,model_10_49_6,0.997136,-0.200556,0.194438,0.989488,0.000338,0.712702,0.564313,0.000411,0.006718,0.018385,1.000556,0.018528,1601.984895,3875.939714,"Hidden Size=[36, 18], regularizer=0.02, learni..."
4,model_10_49_5,0.997094,-0.200457,0.194529,0.989520,0.000343,0.712643,0.564249,0.000410,0.006768,0.018519,1.000565,0.018663,1601.955859,3875.910678,"Hidden Size=[36, 18], regularizer=0.02, learni..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4536,model_7_14_1,-1.198760,-2.480031,0.151564,-15.242932,0.259455,2.065896,0.594347,1.224257,0.084779,0.509367,-3.431876,0.513331,132.698344,319.088083,"Hidden Size=[8, 4], regularizer=0.02, learning..."
4537,model_7_13_6,-1.606354,-2.342459,0.257617,-16.865534,0.307551,1.984228,0.520055,1.455534,0.088147,0.554573,-4.253433,0.558889,132.358226,318.747965,"Hidden Size=[8, 4], regularizer=0.02, learning..."
4538,model_7_22_2,-1.780243,-19.076946,-0.685683,-13.170942,0.328070,11.918542,1.180856,1.388784,0.152486,0.572774,-4.603928,0.577232,132.229054,318.618794,"Hidden Size=[8, 4], regularizer=0.02, learning..."
4539,model_7_14_7,-1.929377,-2.538374,0.190282,-21.311061,0.345668,2.100532,0.567224,1.681622,0.109192,0.587936,-4.904526,0.592511,132.124551,318.514290,"Hidden Size=[8, 4], regularizer=0.02, learning..."
